#### Single Game Entry Strategy:

- **Stars:** Out of the 4-5 likely star players, choose 3.

- **Peasants:** Out of the 5-10 non-star players, choose 5.



In [4]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import filedialog
from datetime import datetime
import xlsxwriter as xl
import re
import itertools
sport = "NHL"
# sport = "NBA"
csv_date = 'Nov4'

def print_current_df(dataframe):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(dataframe)

In [5]:
original_df = pd.read_csv(f'csv21/{sport}_{csv_date}.csv')
print_current_df(original_df)

,ID,First Name,Last Name,Position,Team,Opponent,Game,Time,Salary,FPPG,Injury Status,Starting
0,nhl.p.3789,Marc,Staal,D,DET,BOS,DET@BOS,7:00PM EDT,10,4.1,,
1,nhl.p.3936,Thomas,Greiss,G,DET,BOS,DET@BOS,7:00PM EDT,28,10.6,,No
2,nhl.p.4245,Sam,Gagner,RW,DET,BOS,DET@BOS,7:00PM EDT,11,3.4,,
3,nhl.p.4696,Nick,Leddy,D,DET,BOS,DET@BOS,7:00PM EDT,14,2.6,,
4,nhl.p.5388,Vladislav,Namestnikov,LW,DET,BOS,DET@BOS,7:00PM EDT,12,4.8,,
5,nhl.p.5927,Danny,DeKeyser,D,DET,BOS,DET@BOS,7:00PM EDT,10,2.7,,
6,nhl.p.6013,Adam,Erne,LW,DET,BOS,DET@BOS,7:00PM EDT,11,4.4,,
7,nhl.p.6038,Tyler,Bertuzzi,LW,DET,BOS,DET@BOS,7:00PM EDT,18,12.3,,
8,nhl.p.6346,Jordan,Oesterle,D,DET,BOS,DET@BOS,7:00PM EDT,10,2.9,,
9,nhl.p.6381,Dylan,Larkin,C,DET,BOS,DET@BOS,7:00PM EDT,18,10.4,DTD,


In [6]:
if sport == "NHL":
    df = original_df[['Position', 'Team' ,'Opponent','FPPG','Salary','Injury Status','Starting']].copy()
    df.insert(1, 'Name',original_df["First Name"] + ' ' + original_df["Last Name"])
    df['Value'] = df['FPPG']/df['Salary']
else:
    number_of_teams = len(original_df['Team'].value_counts())
    if(number_of_teams == 2):
        df = original_df[['FPPG','Salary']]
        df.insert(0, 'Name',original_df["First Name"] + ' ' + original_df["Last Name"])
    else:    
        df = original_df[['Position', 'Team', 'Opponent','FPPG','Salary']]
        df.insert(loc=1, column='Pos', value=0)
        conditions = [
            (df['Position'] == 'PG'),
            (df['Position'] == 'SG'),
            (df['Position'] == 'SF'),
            (df['Position'] == 'PF'),
            (df['Position'] == 'C')
        ]
        values = [1, 2, 3, 4, 5]
        df.loc[:,'Pos'] = np.select(conditions, values)
        df.head()
        df.insert(4, 'Name',original_df["First Name"] + ' ' + original_df["Last Name"])


print_current_df(df)

,Position,Name,Team,Opponent,FPPG,Salary,Injury Status,Starting,Value
0,D,Marc Staal,DET,BOS,4.1,10,,,0.410000
1,G,Thomas Greiss,DET,BOS,10.6,28,,No,0.378571
2,RW,Sam Gagner,DET,BOS,3.4,11,,,0.309091
3,D,Nick Leddy,DET,BOS,2.6,14,,,0.185714
4,LW,Vladislav Namestnikov,DET,BOS,4.8,12,,,0.400000
5,D,Danny DeKeyser,DET,BOS,2.7,10,,,0.270000
6,LW,Adam Erne,DET,BOS,4.4,11,,,0.400000
7,LW,Tyler Bertuzzi,DET,BOS,12.3,18,,,0.683333
8,D,Jordan Oesterle,DET,BOS,2.9,10,,,0.290000
9,C,Dylan Larkin,DET,BOS,10.4,18,DTD,,0.577778


In [7]:
if sport == 'NHL': 
    dt = datetime.now()
    today = dt.strftime("%d-%m-%Y")
    file_name = f'{sport}_output_data_{today}.xlsx'
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(file_name, engine='xlsxwriter')

    # Convert the dataframe to an XlsxWriter Excel object. Turn off the default
    # header and index and skip one row to allow us to insert a user defined
    # header.

    df.to_excel (writer, index = False, header=True, sheet_name = "Sheet1")

    # Get the xlsxwriter workbook and worksheet objects.
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    # Get the dimensions of the dataframe.
    (max_row, max_col) = df.shape

    # Create a list of column headers, to use in add_table().
    column_settings = []
    for header in df.columns:
        column_settings.append({'header': header})

    # Add the table.
    worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})

    # Make the columns wider for clarity.
    worksheet.set_column(0, max_col - 1, 12)


    #if sport == 'NHL': 
    superstar_df = df.loc[df['Salary'] > 18]
    superstar_df.to_excel (writer, index = False, header=True, sheet_name = "Superstars")
    worksheet2 = writer.sheets['Superstars']

    (max_row, max_col) = df.shape
    print_current_df(superstar_df)
    worksheet2.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})

    # Make the columns wider for clarity.
    worksheet2.set_column(0, max_col - 1, 12)

    # Close the Pandas Excel writer and output the Excel file.
    writer.close()


,Position,Name,Team,Opponent,FPPG,Salary,Injury Status,Starting,Value
1,G,Thomas Greiss,DET,BOS,10.6,28,,No,0.378571
11,G,Alex Nedeljkovic,DET,BOS,9.6,27,,Yes,0.355556
24,LW,Jakub Vrana,DET,BOS,6.2,19,IR,,0.326316
25,C,Patrice Bergeron,BOS,DET,4.9,27,,,0.181481
26,LW,Brad Marchand,BOS,DET,7.9,30,,,0.263333
33,G,Linus Ullmark,BOS,DET,14.1,32,,No,0.440625
38,RW,David Pastrnak,BOS,DET,7.6,32,,,0.237500
44,G,Jeremy Swayman,BOS,DET,7.1,27,,Yes,0.262963
51,G,Brian Elliott,TB,TOR,3.0,23,,No,0.130435
54,C,Steven Stamkos,TB,TOR,10.3,30,,,0.343333


### Interface

In [5]:
def add_to_lineup(df, player_df, name):
    #if player_df['Name'].str.lower().str.contains(name).any() != True:
    #if player_df['Name'].str.contains("(?i)"+name).any() != True:
    if player_df['Name'].str.lower().str.contains(name.lower()).any() != True:
        player_df = player_df.append(df.loc[df['Name'].str.lower() == name.lower()])
    else:
        print(name + " is already in list")    
    return player_df
def remove_from_lineup(player_df, name):
    print("name? " + name)
    if player_df['Name'].str.lower().str.contains(name.lower()).any() == True:
        player_df.drop(player_df.loc[player_df['Name'].str.lower()==name.lower()].index, inplace=True)
    else:
        print(name + " is not in in list")    
    return player_df

def add_valid_names(df, player_df, names):
    
    return player_df

In [6]:
player_df= pd.DataFrame(columns = df.columns)
player_df['Worst'] = ' '
player_df['Best'] = ' '
if sport == 'NBA':
    player_df['Probable'] = ' '
player_df['Actual'] = ' '
if sport == 'NBA':
    player_df['Deviation'] = ' '
    player_df['Value'] = 1
    player_df['Actual Value'] = ' '

In [7]:
## Trying to lower case
# query = 'donovan mitchell, Alec Burks'
# names = [x.strip() for x in query.split(',')]
# for name in names:
#     if df['Name'].str.contains(name).any():
#         player_df = add_to_lineup(df,player_df,name)    
#     else:
#         print(name + " is not in the list. Please try again")
# print_current_df(player_df)

In [8]:
df2 = df.sort_values(by=['Salary'], ascending = False)
if sport == 'NHL':
    df2 = df2.drop(df2[df2.Starting == "No"].index)
for name in df2['Name']: 
    print(name, end = ", ")
    



Kevin Durant, Luka Doncic, Nikola Jokic, Anthony Davis, Domantas Sabonis, Russell Westbrook, Paul George, James Harden, LeBron James, Damian Lillard, Jimmy Butler, De'Aaron Fox, Malcolm Brogdon, Bam Adebayo, Brandon Ingram, LaMelo Ball, Jonas Valanciunas, CJ McCollum, Caris LeVert, Miles Bridges, Harrison Barnes, Jusuf Nurkic, Evan Mobley, Kristaps Porzingis, Terry Rozier, Gordon Hayward, Darius Garland, Michael Porter Jr., Myles Turner, Jarrett Allen, Collin Sexton, Buddy Hield, Richaun Holmes, Kyle Lowry, Tyrese Haliburton, Nickeil Alexander-Walker, Mason Plumlee, Tyler Herro, Norman Powell, T.J. McConnell, Eric Bledsoe, Reggie Jackson, Will Barton, Ricky Rubio, Kelly Oubre Jr., Devonte' Graham, Chris Duarte, Kevin Love, PJ Washington, Tim Hardaway Jr., Marvin Bagley III, Ivica Zubac, Josh Hart, Robert Covington, Monte Morris, Lauri Markkanen, Terance Mann, Aaron Gordon, Davion Mitchell, Larry Nance Jr., Dorian Finney-Smith, Isaac Okoro, Marcus Morris Sr., Blake Griffin, Guillermo He

In [9]:
name = 'T.J. McConnell'
df['Name'].str.lower().str.contains(name.lower()).any()


True

### Table Add

In [10]:
def get_names(names):
    return [x.strip() for x in names.split(',')]

exit = False


while exit!=True:
    query = input('Name Of Player:')
    if query != 'exit':
        if re.search('^remove', query):
            names_to_remove = re.findall('(?<=remove).*$', query)[0]
            names = get_names(names_to_remove)
            for name in names:
                name = name.title()
                player_df = remove_from_lineup(player_df,name)
                print(f"removed {name}")

        else:
            
            print_current_df(player_df)
            names = get_names(query)
            for name in names:
                name = name.title()
                if df['Name'].str.lower().str.contains(name.lower()).any():
                    player_df = add_to_lineup(df,player_df,name)    
                else:
                    print(name + " is not in the csv list. Please try again")
        print_current_df(player_df)

    else:
        exit = True
player_df = player_df.replace(np.nan, '', regex=True)
player_df

Name Of Player:Kevin Durant, Luka Doncic, Russell Westbrook, Paul George, Jimmy Butler, T.J. McConnell, Brandon Ingram, Jonas Valanciunas,CJ McCollum


,Position,Pos,Team,Opponent,Name,FPPG,Salary,Worst,Best,Probable,Actual,Deviation,Value,Actual Value


,Position,Pos,Team,Opponent,Name,FPPG,Salary,Worst,Best,Probable,Actual,Deviation,Value,Actual Value
19,SF,3,BKN,IND,Kevin Durant,52.9,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,PG,1,DAL,DEN,Luka Doncic,46.0,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,PG,1,LAL,CLE,Russell Westbrook,43.8,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,SF,3,LAC,POR,Paul George,48.6,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,SF,3,MIA,CHA,Jimmy Butler,47.5,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PG,1,IND,BKN,T.J. McConnell,18.0,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,SF,3,NO,SAC,Brandon Ingram,40.8,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,C,5,NO,SAC,Jonas Valanciunas,44.2,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189,SG,2,POR,LAC,CJ McCollum,44.4,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Name Of Player:Miles Bridges


,Position,Pos,Team,Opponent,Name,FPPG,Salary,Worst,Best,Probable,Actual,Deviation,Value,Actual Value
19,SF,3,BKN,IND,Kevin Durant,52.9,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,PG,1,DAL,DEN,Luka Doncic,46.0,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,PG,1,LAL,CLE,Russell Westbrook,43.8,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,SF,3,LAC,POR,Paul George,48.6,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,SF,3,MIA,CHA,Jimmy Butler,47.5,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PG,1,IND,BKN,T.J. McConnell,18.0,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,SF,3,NO,SAC,Brandon Ingram,40.8,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,C,5,NO,SAC,Jonas Valanciunas,44.2,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189,SG,2,POR,LAC,CJ McCollum,44.4,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN


...

Name Of Player:Bruce Brown


,Position,Pos,Team,Opponent,Name,FPPG,Salary,Worst,Best,Probable,Actual,Deviation,Value,Actual Value
19,SF,3,BKN,IND,Kevin Durant,52.9,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,PG,1,DAL,DEN,Luka Doncic,46.0,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,PG,1,LAL,CLE,Russell Westbrook,43.8,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,SF,3,LAC,POR,Paul George,48.6,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,SF,3,MIA,CHA,Jimmy Butler,47.5,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PG,1,IND,BKN,T.J. McConnell,18.0,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,SF,3,NO,SAC,Brandon Ingram,40.8,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,C,5,NO,SAC,Jonas Valanciunas,44.2,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189,SG,2,POR,LAC,CJ McCollum,44.4,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,PF,4,CHA,MIA,Miles Bridges,43.9,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Position,Pos,Team,Opponent,Name,FPPG,Salary,Worst,Best,Probable,Actual,Deviation,Value,Actual Value
19,SF,3,BKN,IND,Kevin Durant,52.9,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,PG,1,DAL,DEN,Luka Doncic,46.0,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,PG,1,LAL,CLE,Russell Westbrook,43.8,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,SF,3,LAC,POR,Paul George,48.6,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,SF,3,MIA,CHA,Jimmy Butler,47.5,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PG,1,IND,BKN,T.J. McConnell,18.0,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,SF,3,NO,SAC,Brandon Ingram,40.8,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,C,5,NO,SAC,Jonas Valanciunas,44.2,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189,SG,2,POR,LAC,CJ McCollum,44.4,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,PF,4,CHA,MIA,Miles Bridges,43.9,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Name Of Player:exit


,Position,Pos,Team,Opponent,Name,FPPG,Salary,Worst,Best,Probable,Actual,Deviation,Value,Actual Value
19,SF,3,BKN,IND,Kevin Durant,52.9,52,,,,,,,
45,PG,1,DAL,DEN,Luka Doncic,46.0,52,,,,,,,
90,PG,1,LAL,CLE,Russell Westbrook,43.8,46,,,,,,,
171,SF,3,LAC,POR,Paul George,48.6,46,,,,,,,
123,SF,3,MIA,CHA,Jimmy Butler,47.5,41,,,,,,,
5,PG,1,IND,BKN,T.J. McConnell,18.0,20,,,,,,,
156,SF,3,NO,SAC,Brandon Ingram,40.8,36,,,,,,,
153,C,5,NO,SAC,Jonas Valanciunas,44.2,34,,,,,,,
189,SG,2,POR,LAC,CJ McCollum,44.4,33,,,,,,,
107,PF,4,CHA,MIA,Miles Bridges,43.9,30,,,,,,,


In [11]:

dt = datetime.now()
now = dt.strftime("%Y-%m-%d %H_%M")

file_name = f'output/lineup_{now}.xlsx'
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object. Turn off the default
# header and index and skip one row to allow us to insert a user defined
# header.

player_df.to_excel (writer, index = False, header=True)

# Get the xlsxwriter workbook and worksheet objects.
workbook = writer.book
worksheet = writer.sheets['Sheet1']

# Get the dimensions of the dataframe.
(max_row, max_col) = player_df.shape

# Create a list of column headers, to use in add_table().
column_settings = []
for header in player_df.columns:
    column_settings.append({'header': header})

# Add the table.
worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})

# Make the columns wider for clarity.
worksheet.set_column(0, max_col - 1, 12)
# for row in range(2,player_df.shape[0]+2):
#     formula = f'=K{row}-J{row}'
#     worksheet.write_formula(f"L{row}", formula)
# for row in range(2,player_df.shape[0]+2):
#     formula = f'=J{row}/G{row}'
#     worksheet.write_formula(f"M{row}", formula)
# for row in range(2,player_df.shape[0]+2):
#     formula = f'=K{row}/G{row}'
#     worksheet.write_formula(f"N{row}", formula)

# Close the Pandas Excel writer and output the Excel file.
writer.close()


In [12]:
player_df['Name']
for name in player_df['Name']:
    print(name, end = ", ")

Kevin Durant, Luka Doncic, Russell Westbrook, Paul George, Jimmy Butler, T.J. McConnell, Brandon Ingram, Jonas Valanciunas, CJ McCollum, Miles Bridges, Harrison Barnes, Maxi Kleber, Gordon Hayward, Jarrett Allen, Collin Sexton, Richaun Holmes, Tyrese Haliburton, Nickeil Alexander-Walker, Mason Plumlee, Tyler Herro, Reggie Jackson, Will Barton, Ricky Rubio, Devonte' Graham, Kevin Love, Lauri Markkanen, Dorian Finney-Smith, Jalen McDaniels, Jalen Brunson, Bruce Brown, 

In [13]:
#file_name='output/lineup_2021-07-01 17_32.xlsx'
filled_df= pd.read_excel(file_name)
print_current_df(filled_df)

,Position,Pos,Team,Opponent,Name,FPPG,Salary,Worst,Best,Probable,Actual,Deviation,Value,Actual Value
0,SF,3,BKN,IND,Kevin Durant,52.9,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PG,1,DAL,DEN,Luka Doncic,46.0,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PG,1,LAL,CLE,Russell Westbrook,43.8,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SF,3,LAC,POR,Paul George,48.6,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SF,3,MIA,CHA,Jimmy Butler,47.5,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PG,1,IND,BKN,T.J. McConnell,18.0,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,SF,3,NO,SAC,Brandon Ingram,40.8,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,C,5,NO,SAC,Jonas Valanciunas,44.2,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,SG,2,POR,LAC,CJ McCollum,44.4,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,PF,4,CHA,MIA,Miles Bridges,43.9,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Single-game analysis

In [14]:
single_df = filled_df[['Name','Salary','Worst','Best','Probable']]
print_current_df(single_df)
create_lineup(single_df)

,Name,Salary,Worst,Best,Probable
0,Kevin Durant,52,NaN,NaN,NaN
1,Luka Doncic,52,NaN,NaN,NaN
2,Russell Westbrook,46,NaN,NaN,NaN
3,Paul George,46,NaN,NaN,NaN
4,Jimmy Butler,41,NaN,NaN,NaN
5,T.J. McConnell,20,NaN,NaN,NaN
6,Brandon Ingram,36,NaN,NaN,NaN
7,Jonas Valanciunas,34,NaN,NaN,NaN
8,CJ McCollum,33,NaN,NaN,NaN
9,Miles Bridges,30,NaN,NaN,NaN


NameError: name 'create_lineup' is not defined

In [ ]:
max_salary = 30

In [ ]:
## best 5 probable, best 5 worst, best 5 best. Create combinations of lineups with the most recurring names

def create_lineup(df):
    lineup = []
    df2 = df.ge()
        
    
    print(highest_paid_player.Name)
    

In [ ]:
numbers = [1, 2, 3, 7, 7, 9, 10]
result = [seq for i in range(len(numbers), 0, -1) for seq in itertools.combinations(numbers, i) if sum(seq) >= 10]
print(result)

In [ ]:
class Lineup: 
    superstar = ""
    megastar = ""
    star = ""
    other_players = []   
    

In [ ]:
class MyClass:
    """A simple example class"""
    i = 12345

    def __init__(self,data):
        self.data = data
l = [1,2,"e"]
x = MyClass(l)
x.data


### Multi-game analysis

In [ ]:
# sorted_df = filled_df.sort_values(by=['Pos','Probable'],ascending=[True,False]).drop(['Notes', 'Pts', 'Reb', 'Ast', 'Stl', 'Blk', 'TO',
#        'Total'],axis = 1)

# # sorted_df.insert(12,'Worst Value',sorted_df["Worst"] / sorted_df["Salary"])
# # sorted_df.insert(13,'Best Value',sorted_df["Best"] / sorted_df["Salary"])
# df=sorted_df
# lineups = []
# positions = []



# df = df.loc[:, lambda df: ['Pos','Salary','Name', "Worst","Best",'Probable']]
# [positions.append(df.loc[(sorted_df['Pos'] == i)].sort_values(by=['Pos','Probable'],ascending=[True,False])) for i in range(1,6)]

# [print_current_df(position_df) for position_df in positions]

In [ ]:
def select_eligible(df):
    = sorted_df.loc[(sorted_df['Salary'] > 30)]
df

In [ ]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))


In [ ]:
# d = pd.concat(
#     [df.loc[a].reset_index(), df.loc[b].reset_index()],
#     keys=['a', 'b'], axis=1
# )

# d
